In [1]:
import pandas as pd
import numpy as np

# Decision Tree - 14/11/2023

- Import des données

In [2]:
path = './data'
x_data = pd.read_csv(path+'/Xtrain_hgcGIrA.csv', sep=',') # features
y_data = pd.read_csv(path+'/Ytrain_yL5OjS4.csv', sep=',', usecols=[1]) # occupancy rate
x_predict = pd.read_csv(path+'/Xtest.csv', sep=',') # features pour la prédiction

- Mise en forme de x_data

In [3]:
x_data=x_data.drop('date', axis=1) # on supprime la colonne date
x_data=x_data.drop('way', axis=1) # on supprime la colonne way
x_data["hour"] = x_data["hour"].apply(lambda x: int(x[:2]) if isinstance(x, str) else np.nan) # on transforme les strings des heures en float
x_data["station"] = x_data["station"].astype('category')
cat_columns = x_data.select_dtypes(["category"]).columns
x_data[cat_columns] = x_data[cat_columns].apply(lambda x: x.cat.codes)

- Mise en forme de x_predict

In [4]:
x_predict=x_predict.drop('date', axis=1) # on supprime la colonne date
x_predict=x_predict.drop('way', axis=1) # on supprime la colonne way
x_predict["hour"] = x_predict["hour"].apply(lambda x: int(x[:2]) if isinstance(x, str) else np.nan) # on transforme les strings des heures en float
x_predict["station"] = x_predict["station"].astype('category')
cat_columns = x_predict.select_dtypes(["category"]).columns
x_predict[cat_columns] = x_predict[cat_columns].apply(lambda x: x.cat.codes)

- Apprentissage du modèle retenu

In [6]:
from sklearn.tree import DecisionTreeRegressor

In [7]:
DR_VF = DecisionTreeRegressor(max_depth=194, min_samples_split=0.00035714285714285714, criterion='friedman_mse', max_features=9, random_state=11)
DR_VF.fit(x_data.values, y_data.values)
DR_VF.score(x_data.values, y_data.values)

0.9200147463817054

- Prédiction et export

In [8]:
y_predict = DR_VF.predict(x_predict.values)

In [9]:
from export import export_to_csv

export_to_csv(y_predict, "decision_tree_23_11_14")

Score obtenu : 0,03335999746890835

# Bagging Regressor with Decision Tree - 14/11/2023

- Import des données

In [10]:
path = './data'
x_data = pd.read_csv(path+'/Xtrain_hgcGIrA.csv', sep=',') # features
y_data = pd.read_csv(path+'/Ytrain_yL5OjS4.csv', sep=',', usecols=[1]) # occupancy rate
x_predict = pd.read_csv(path+'/Xtest.csv', sep=',') # features pour la prédiction

- Mise en forme de x_data

In [11]:
x_data=x_data.drop('date', axis=1) # on supprime la colonne date
x_data=x_data.drop('way', axis=1) # on supprime la colonne way
x_data["hour"] = x_data["hour"].apply(lambda x: int(x[:2]) if isinstance(x, str) else np.nan) # on transforme les strings des heures en float
x_data["station"] = x_data["station"].astype('category')
cat_columns = x_data.select_dtypes(["category"]).columns
x_data[cat_columns] = x_data[cat_columns].apply(lambda x: x.cat.codes)

- Mise en forme de x_predict

In [12]:
x_predict=x_predict.drop('date', axis=1) # on supprime la colonne date
x_predict=x_predict.drop('way', axis=1) # on supprime la colonne way
x_predict["hour"] = x_predict["hour"].apply(lambda x: int(x[:2]) if isinstance(x, str) else np.nan) # on transforme les strings des heures en float
x_predict["station"] = x_predict["station"].astype('category')
cat_columns = x_predict.select_dtypes(["category"]).columns
x_predict[cat_columns] = x_predict[cat_columns].apply(lambda x: x.cat.codes)

- Apprentissage du modèle retenu

In [13]:
from sklearn.ensemble import BaggingRegressor

In [14]:
BR_1 = BaggingRegressor(
        DecisionTreeRegressor(max_depth=194, min_samples_split=0.00035714285714285714, criterion='friedman_mse', max_features=9, random_state=11),
        n_estimators=52)
BR_1.fit(x_data.values, y_data.values)
BR_1.score(x_data.values, y_data.values)

/home/mcoutier/miniconda3/envs/STASC/lib/python3.10/site-packages/sklearn/ensemble/_bagging.py:509: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)


0.9228845942072106

- Prédiction et export

In [15]:
y_predict = BR_1.predict(x_predict.values)
export_to_csv(y_predict, "bagging_decision_tree_23_11_14")

Score obtenu : 0,0337084999304966

Ce score est inférieur à celui du DecisionTree ci-dessus. Cela peut arriver. Bagging Regressor n'est pas forcément plus performant mais il est plus robuste.